In [1]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

from typing import Any, Optional, List

import qubx
%qubxd dev 

%load_ext autoreload
%autoreload 2

from qubx import lookup, logger
from qubx.core.strategy import IStrategy, StrategyContext, TriggerEvent, PositionsTracker
from qubx.pandaz.utils import *

# - - - - - - - - - - - - - - - - - - - - - - - -
from qubx.trackers import PortfolioRebalancerTracker
from qubx.core.series import Quote
from qubx.ta.indicators import sma, ema
from qubx.data.readers import CsvStorageDataReader, MultiQdbConnector, AsTimestampedRecords, AsQuotes, RestoreTicksFromOHLC, AsPandasFrame, RestoreTicksFromOHLC, AsOhlcvSeries
from qubx.core.basics import Deal, Instrument, Order, Position, Signal
from qubx.core.utils import recognize_time
from qubx.utils.runner import get_account_config

def Q(time: str, bid: float, ask: float) -> Quote:
    return Quote(recognize_time(time), bid, ask, 0, 0)

# r = CsvStorageDataReader("../tests/data/csv")
# r = MultiQdbConnector('xlydian-data')

from qubx import QubxLogConfig
from qubx.backtester.simulator import simulate
from qubx.core.metrics import tearsheet

 >  [dev] installing cython rebuilding hook

⠀⠀⡰⡖⠒⠒⢒⢦⠀⠀   
⠀⢠⠃⠈⢆⣀⣎⣀⣱⡀  QUBX | Quantitative Backtesting Environment 
⠀⢳⠒⠒⡞⠚⡄⠀⡰⠁         (c) 2024, ver. Dev
⠀⠀⠱⣜⣀⣀⣈⣦⠃⠀⠀⠀ 
        
2024-07-27 10:44:47.338 [ ℹ️ ] NTP offset controller thread is started


2024-07-27 11:21:26.253 [ ⚠️ ] qubx.utils.ntp:__correct_offset:28 - time.windows.com NTP server request exception: No response received from time.windows.com.
2024-07-27 20:46:06.405 [ ⚠️ ] qubx.utils.ntp:__correct_offset:28 - time.windows.com NTP server request exception: [Errno 11001] getaddrinfo failed


# Strat for signals testing

In [17]:
class StrategyForTracking(IStrategy):
    timeframe: str = "1Min"
    fast_period = 5
    slow_period = 12

    def on_fit(self, ctx: "StrategyContext", fit_time: str | pd.Timestamp, previous_fit_time: str | pd.Timestamp | None = None):
        logger.info(f" -> Fit is called | fit_time: {fit_time} / prev: {previous_fit_time}")

    def on_event(self, ctx: StrategyContext, event: TriggerEvent) -> List[Signal] | None:
        for i in ctx.instruments:
            # logger.info(f"\t{i.symbol} : {ctx.broker_provider.get_quote(i.symbol)}")
            ohlc = ctx.ohlc(i, self.timeframe)
            fast = sma(ohlc.close, self.fast_period)
            slow = sma(ohlc.close, self.slow_period)
            pos = ctx.positions[i.symbol].quantity

            if pos <= 0:
                if (fast[0] > slow[0]) and (fast[1] < slow[1]):
                    ctx.trade(i, abs(pos) + i.min_size * 10)

            if pos >= 0:
                if (fast[0] < slow[0]) and (fast[1] > slow[1]):
                    ctx.trade(i, -pos - i.min_size * 10)

        return None

    def ohlcs(self, timeframe: str) -> Dict[str, pd.DataFrame]:
        return {s.symbol: self.ctx.ohlc(s, timeframe).pd() for s in self.ctx.instruments}

    def tracker(self, ctx: StrategyContext) -> PositionsTracker:
        return PortfolioRebalancerTracker(1000, 0).set_context(ctx)

In [12]:
r = MultiQdbConnector('xlydian-data')

2024-07-27 10:49:44.102 [ 🐞 ] Connected to QuestDB at xlydian-data:8812


In [ ]:
r = CsvStorageDataReader("../tests/data/csv")

QubxLogConfig.set_log_level("DEBUG")

rep = simulate({
    
        "As Strategy": StrategyForTracking(
            timeframe="5Min", fast_period=5, slow_period=15),
    },
    r, 10000, ["BINANCE.UM:BTCUSDT"], 
    dict(type="ohlc", timeframe="1Min", nback=0),
    "5Min -1Sec",
    "vip0_usdt", 
    "2024-01-01", "2024-01-02", 
)

In [23]:
r1 = rep[0]

In [24]:
r1.portfolio_log

,BTCUSDT_PnL,BTCUSDT_Pos,BTCUSDT_Price,BTCUSDT_Value,BTCUSDT_Commissions
timestamp,,,,,
2024-01-01 00:00:00,0.000,0.00,0.0,0.000,0.0
2024-01-01 00:05:00,0.000,0.00,0.0,0.000,0.0
2024-01-01 00:10:00,0.000,0.00,0.0,0.000,0.0
2024-01-01 00:15:00,0.000,0.00,0.0,0.000,0.0
2024-01-01 00:20:00,0.000,0.00,0.0,0.000,0.0
...,...,...,...,...,...
2024-01-01 23:40:00,0.941,0.01,43713.2,441.720,0.0
2024-01-01 23:45:00,-0.321,0.01,43713.2,441.399,0.0
2024-01-01 23:50:00,-0.153,0.01,43713.2,441.246,0.0


In [25]:
tearsheet(rep)